In [1]:
import numpy as np
import math
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import scipy as sp
from scipy.integrate import odeint
from scipy.integrate import solve_ivp
from scipy.optimize import least_squares as ls

%matplotlib inline
mpl.rcParams['figure.dpi']=75
plt.rcParams['svg.fonttype'] = 'none'
plt.rcParams.update({'font.family':'sans-serif'})
plt.rcParams.update({'font.sans-serif':'Arial'})

In [2]:
def monopH(t,y, *args):
    K = args[0]
    pref = args[1]
    sigma = args[2]
    delta = args[3]
    gamma = args[4]
    mu = args[5]
    kdead = args[6]
    
    
    dead = y[0]
    od = y[1]
    pH = y[2]
    
    
    ddead = -kdead*max(0,dead)
    dod = (od*(mu-od/K))*(np.exp(-(pH-pref)**2/sigma**2)-delta)+kdead*max(0,dead)
    dpH = gamma*od
    
    
    return [ddead, dod, dpH]    


In [4]:
pHdf = pd.ExcelFile('All_pH.xlsx')
ODdf = pd.ExcelFile('All_OD.xlsx')

allpH = [4,5,6,7,8,9]

allspecies = ['AC','BA','BC','BL','BT','BV','CC','DF','wt','delarc','parc-','parc+']

In [ ]:
allparams = []
allMSE = []
for species in allspecies:
    if species in ['AC','BA','BC','BL','BT','BV','CC','DF',]:
        reps = 4
    else:
        reps = 3
    
    speciesDF = pd.read_excel(ODdf,species)
    AllMin=[]
    exp_OD=[]
    exp_t = list(speciesDF['Time'])    
    avg_OD=[]
    for j in range(6):
        Min=min(min(speciesDF[str(allpH[j])+'_1']),min(speciesDF[str(allpH[j])+'_2']),min(speciesDF[str(allpH[j])+'_3']))
        AllMin.append(speciesDF[str(allpH[j])+'_'+str(1)][0]-Min+0.01)
        average_OD=[(a+b+c+d)/4 for a,b,c,d in zip(np.array(speciesDF[str(allpH[j])+'_'+str(1)])-Min+0.01,np.array(speciesDF[str(allpH[j])+'_'+str(2)])-Min+0.01,np.array(speciesDF[str(allpH[j])+'_'+str(3)])-Min+0.01,np.array(speciesDF[str(allpH[j])+'_'+str(4)])-Min+0.01)]
        avg_OD.extend(average_OD)
        for i in range(reps):
            exp_OD.append(np.array(speciesDF[str(allpH[j])+'_'+str(i+1)])-Min+0.01)
    
    pHspeciesDF = pd.read_excel(pHdf,species)
    exp_Hplus=[]
    exp_pH=[]
    AllstartpH=[]
    avg_pH=[]
    for j in range(6):
        AllstartpH.append(pHspeciesDF[str(allpH[j])+'_1'][0])
        average_pH=[(a+b+c+d)/4 for a,b,c,d in zip(np.array(pHspeciesDF[str(allpH[j])+'_'+str(1)]),np.array(pHspeciesDF[str(allpH[j])+'_'+str(2)]),np.array(pHspeciesDF[str(allpH[j])+'_'+str(3)]),np.array(pHspeciesDF[str(allpH[j])+'_'+str(4)]))]
        avg_pH.extend(average_pH)
        for i in range(reps):
            exp_pH.append(np.array(pHspeciesDF[str(allpH[j])+'_'+str(i+1)]))
    
    def residuals(params):
        params=tuple(params)
        res=np.zeros(6)
        for j in range(6):
            sim_P=solve_ivp(monopH, t_span=[0,24],y0=[AllMin[j],0,AllstartpH[j]],t_eval=[0,3,6,9,12,24],args = params)
            sim_OD=sim_P.y[1]
            averageOD = sum(sim_OD)/len(sim_OD)
            sim_pH=sim_P.y[2]
            averagepH = sum(sim_pH)/len(sim_pH)  
            res += [((a - b))**2 for a, b in zip(sim_OD,avg_OD[j*6:j*6+6])]
            res += [((a - b))**2 for a, b in zip(sim_pH,avg_pH[j*6:j*6+6])]
        return res

    initial_guess = (10,7,1,1,-0.01,1,10)
    low_bounds = [0,4,0,0,-10,0,0]
    up_bounds = [100,9,5,10,10,10,100]

    result = ls(residuals, initial_guess, bounds=(low_bounds, up_bounds))
    fitted_params = result.x
    MSE = result.cost
    
    allparams.append(fitted_params)
    allMSE.append(MSE)



In [ ]:
allparams

In [ ]:
allMSE